In [6]:
from transformers import BertTokenizer, BertModel
import numpy as np
import torch
import math

def bert_text_preprocessing(_text, _tokenizer):
    # adds bert-specific tokens
    tagged_text = "[CLS] " + _text + " [SEP]"
    # tokenize text
    tokenized_text = tokenizer.tokenize(tagged_text)
    # token to ids and token to segment ids
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = np.array([1] * len(indexed_tokens))
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    # print(f'tokens_tensor: {tokens_tensor}')
    segments_tensors = torch.tensor([segments_ids])
    # print(f'segments_tensors: {segments_tensors}')

    return tokenized_text, tokens_tensor, segments_tensors


def get_bert_embeddings(_tokens_tensor, _segments_tensors, _model):
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = _model(_tokens_tensor, _segments_tensors)
        print(f'outputs: \n{outputs}')
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

def compute_cosine_similarity(_u, _v):
    denominator = math.sqrt(np.sum(np.array(_u) ** 2)) * math.sqrt(np.sum(np.array(_v) ** 2))
    return np.dot(_u, _v) / denominator

def align_sentences():
    return 0

"""
In this program, we are analyzing the use of the word "set"
in different sentences. The goal is to compare the usage of the word
in the sentences using BERT embedding, and for each sentence, find
a sample sentence where the usage of "set" is the closest.
"""

texts = ["set",
         "On your mark, get set, go!",
         "This set of numbers has an interesting property.",
         "When I got to the party, I realized that it was all a set up.",
         "I had set the temperature higher this morning.",
         "These beautiful bowls come as a set of four.",
         "Who set the house on fire?",
         "The story is set in 1960's Paris.",
         'The word "set" has several different meanings.']

target_word_embeddings = []

# loading the pre-trained BERT model (other models: bert-base-multilingual-cased, bert-base-uncased)
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True)
# loading bert tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

for text in texts:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preprocessing(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)

    # Find the position 'set' in list of tokens
    word_index = tokenized_text.index('set')
    # Get the embedding for bank
    word_embedding = list_token_embeddings[word_index]
    target_word_embeddings.append(word_embedding)

list_of_distances = []
for text1, embed1 in zip(texts, target_word_embeddings):
    print(f'\nTARGET: {text1}')
    cos_dists = np.array([])
    for text2, embed2 in zip(texts, target_word_embeddings):
        cos_dist = compute_cosine_similarity(embed1, embed2)
        list_of_distances.append([text1, text2, cos_dist])
        if text2 == text1:
            cos_dists = np.append(cos_dists, 0)
        else:
            cos_dists = np.append(cos_dists, cos_dist)
            print(f'dist: {round(cos_dist, 4)}, text: {text2}')
    print(f'best match: {texts[np.argmax(cos_dists)]}')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

<ipython-input-6-d4fff8b86154>:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  segments_tensors = torch.tensor([segments_ids])


outputs: 
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3614,  0.0659,  0.0546,  ..., -0.2400,  0.0199,  0.3533],
         [ 0.0946, -0.2009,  0.5869,  ..., -0.0017, -0.1964,  0.1412],
         [ 0.8118,  0.0556, -0.3274,  ...,  0.1741, -0.8902, -0.2291]]]), pooler_output=tensor([[-8.0605e-01, -1.6846e-01,  5.9146e-01,  6.3911e-01, -4.3142e-01,
         -1.1491e-01,  8.6804e-01,  2.0222e-01,  2.5895e-01, -9.9961e-01,
          1.1808e-01,  7.1688e-02,  9.6374e-01, -2.4979e-01,  8.5671e-01,
         -3.9237e-01, -4.6366e-02, -4.8368e-01,  4.0513e-01, -7.7283e-01,
          4.7281e-01,  8.1527e-01,  5.8394e-01,  1.8720e-01,  3.5207e-01,
          1.5038e-01, -4.9393e-01,  8.7121e-01,  9.1425e-01,  6.3915e-01,
         -6.3394e-01,  1.9034e-01, -9.6659e-01, -2.2098e-01,  3.8441e-01,
         -9.6784e-01,  1.3078e-01, -6.7602e-01, -3.8175e-02,  9.1101e-03,
         -8.3077e-01,  2.7705e-01,  9.9162e-01,  1.0088e-01, -4.0616e-02,
         -3.2234e-01, -9.9900e-